In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)

In [4]:
print(dataset)

           age         bp  al  su         bgr          bu        sc  \
0     2.000000  76.459948   3   0  148.112676   57.482105  3.077356   
1     3.000000  76.459948   2   0  148.112676   22.000000  0.700000   
2     4.000000  76.459948   1   0   99.000000   23.000000  0.600000   
3     5.000000  76.459948   1   0  148.112676   16.000000  0.700000   
4     5.000000  50.000000   0   0  148.112676   25.000000  0.600000   
..         ...        ...  ..  ..         ...         ...       ...   
394  51.492308  70.000000   0   0  219.000000   36.000000  1.300000   
395  51.492308  70.000000   0   2  220.000000   68.000000  2.800000   
396  51.492308  70.000000   3   0  110.000000  115.000000  6.000000   
397  51.492308  90.000000   0   0  207.000000   80.000000  6.800000   
398  51.492308  80.000000   0   0  100.000000   49.000000  1.000000   

            sod       pot       hrmo  ...  pc_normal  pcc_present  ba_present  \
0    137.528754  4.627244  12.518156  ...          0            0 

In [5]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [6]:
dataset["classification_yes"].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [7]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset[['classification_yes']]

In [8]:
indep.shape

(399, 27)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(indep, dep, test_size=1/3, random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid={'criterion':['gini', 'entropy'],
           'max_features':['auto','sqrt','log2'],'n_estimators':[10,100]}

grid=GridSearchCV(RandomForestClassifier(),param_grid, refit=True, verbose=3, n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_pa

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [12]:
print(grid.best_params_)

{'criterion': 'gini', 'max_features': 'log2', 'n_estimators': 100}


In [13]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, grid_predictions)

In [15]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test, grid_predictions)

In [16]:
print("The Confusion Matrix:\n",cm)

The Confusion Matrix:
 [[51  0]
 [ 1 81]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [18]:
from sklearn.metrics import roc_auc_score
print("Roc_auc_score is:",roc_auc_score(y_test,grid.predict_proba(X_test)[:,1]))

Roc_auc_score is: 0.9997608799617408


In [19]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000000,0.000000,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.000000,0.000000,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.051797,0.004039,0.016246,0.005125,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.945100,0.942166,0.981217,0.981031,0.981031,0.966109,0.018375,7
3,0.259901,0.010069,0.017897,0.004669,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",1.000000,0.942166,0.962573,0.981031,1.000000,0.977154,0.022341,3
4,0.027231,0.004920,0.010741,0.001202,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.963284,0.961755,0.962573,0.962264,1.000000,0.969975,0.015021,6
5,0.252480,0.006722,0.020229,0.000598,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",1.000000,0.942166,0.981217,0.981031,1.000000,0.980883,0.021119,1
6,0.001652,0.002613,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.000000,0.000000,0.000000,0.000000,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.028489,0.003666,0.009465,0.000403,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.945100,0.942166,0.907035,0.962264,0.962264,0.943766,0.020188,8
9,0.258574,0.008376,0.019326,0.002543,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.981569,0.942166,0.981217,0.981031,1.000000,0.977197,0.018958,2


In [20]:
import pickle
filename="model_RF_classification.sav"
pickle.dump(grid,open(filename,'wb'))

In [21]:
loaded_model=pickle.load(open("model_RF_classification.sav",'rb'))      
result=loaded_model.predict([[10,70,2,0,200,50,2,130,4,8,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1]])

In [22]:
result

array([1])